In [14]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import ollama

In [15]:
MODEL = "llama3.2"

In [24]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [25]:
# Let's try one out

ed = Website("https://edwarddonner.com")
# print(ed.title)
# print(ed.text)

In [59]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a detailed summary, ignoring text that might be navigation related. \
Respond in markdown."

In [58]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
                    please provide a detailed summary of this website in markdown. \
                    If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [60]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [61]:
from openai import OpenAI

In [62]:
# Basic and a little complex way to call ollama api
def summarize(url):
    website = Website(url)
    ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
    response = ollama_via_openai.chat.completions.create(
        model=MODEL,
        messages=messages_for(website)
    )
    return response.choices[0].message.content

In [55]:
# Easy and adavanced way
# And now: call the Ollama function instead of OpenAI. this way it autmatically calls the endpoint, this end point was basically built for openAi but now everyone
# uses the same end point so it automatically calls this end point using this python package built by ollama. 

# def summarize(url):
#     website = Website(url)
#     messages = messages_for(website)
#     response = ollama.chat(model=MODEL, messages=messages)
#     return response['message']['content']

In [56]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [63]:
display_summary("https://vullyblues.ch")

# Vully Blues Festival Website Summary
=====================================

## Introduction
-----------------

The Vully Blues Festival website appears to be dedicated to a music festival that showcases blues music and artists.

## Key Sections
---------------

### Events

* The festival details:
	+ Vully Blues Festival 2025: November 7-8, 2025
	+ Des informations suivront dans le courant du mois d'août (information will be provided in the coming weeks of August)
* Concerts de saison (Seasonal Concerts) and Festival details are currently empty

### Artistes et Programme 2025 (Artists and Program 2025)

* Not visible due to website limitations, but links to reserve tickets and follow sponsors are available.

### Infos Pratiques (Practical Information)

* Practical information is not available in the current website content.

### Éditions Passées (Past Editions)

* Vully Blues Festival 2024 details are mentioned

### Soutien (Support)

* BENEVOLES (Volunteers)
	+ Members
	+ Contact us
* MEMBRES (Members)
	+ Members list not visible
* SOUTIEN (Support)
	+ Donors loading...
	+ Sponsors loading...

### Contact

* Website contact information:
	+ Address: Chemin aux Maisonnettes 4, CH-1788 Praz
	+ Email: [not visible]
	+ Follow us (but link is not provided)

## Conclusion
----------

The website seems to be in the process of updating or improving its content. Most sections are currently empty, except for a few festival details and sponsor information.